In [1]:
import pandas as pd

import re


from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

In [5]:
year_folder_id_dict = {'2010':'113DAB_g6P3b-gbrpFog682EfdILcBMt6','2011':'1Yh9610KUMxZLNDiyUOnng-7PbhlYbt3x',
                       '2012':'1TEWAyyxf6dJ3K6hMiliayJ4VV4ilvwIH','2013':'1KWNg6Me11spV-JRFOzwlvNpvoYMLI59C',
                       '2014':'1IgDuaPlc9b-MdP6VELFXcTffqSRHYPU5','2015':'1CrHueAEH76p95tIOryMrdu5nV-qW702M',
                       '2016':'1hpViY56I20A6k6EeQGrOn6QG11j7gb58','2017':'1H2HXwALMyzxBIMJyluYkTHJKplqGIpUF',
                       '2018':'1-jlTY-u_izedKKNenevC3kgM6yaKR7b7','2019':'1Qm-Sse4vbOx2IOAILUCTvXwqShWlynQn'}

events_folder_id_dict = {'2010':'1CPmsZoNqsDslYUDuMMYAiTRdgcbjuIj6','2011':'1FQenIULvAsgQN4sLSpcLdjjfStfeIvga',
                       '2012':'1ABi7lAAmlARbrGlBJv8Tz6IFXVSYzLdR','2013':'1MMg8bUfZj3eZf_gAlr3k2N9_LWmO5DWr',
                       '2014':'135rCpr_1_N00nmsKOcyWjmQPmT_n8I96','2015':'1Gjm0qtTXL-21qNQJtloxNGtJq3Q1Xlyq',
                       '2016':'1qRUeOe4mFx5ofuYHxiqL4u1HvqxLCz96','2017':'15DY5sNvP8O2jgus7Qythy8Nzeeo7cZAI',
                       '2018':'1AKHSz-t3vkUsLE2IC4TdrMUznjNC3YpU','2019':'1qOX3DQFtU_j4czyUMndjpn9c0crZ8ue1'}
events_folder_id = '165-gELn98fyKGaWnmWSkJuUuz1aj8XMg'

In [2]:
df = pd.read_csv('df_results.csv', index_col=0)

In [6]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)

In [7]:
saved_files = []
for folder_id in year_folder_id_dict.values():
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
    files = results.get("files", [])
    next_page_token = results.get("nextPageToken")
    while next_page_token:
        results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
        files.extend(results.get("files", []))
        next_page_token = results.get("nextPageToken")
    for file in files:
        saved_files.append(file.get('name').replace('.pdf',''))
print(len(saved_files))

2812


In [8]:
mask = df['date'].isin(saved_files)
df.loc[mask, 'GD'] = 'raw'

In [9]:
mask = ~df['date'].isin(saved_files)
df.loc[mask, 'GD'] = 'missed'

**Number of saved files in Raw_data folder**

In [10]:
df['GD'].value_counts()

raw       2813
missed     115
Name: GD, dtype: int64

In [30]:
with open('missing_dates.txt', 'r') as f:
    missing_dates = [x.strip() for x in f.read().split('\n')]

In [43]:
mask = (~df['date'].isin(missing_dates)) & (df['GD']=='missed')
df_missing_files = df.loc[mask]
df_missing_files.to_csv('df_missing_files.csv')

In [11]:
df

,issue,url,date,pdf_link,status,GD
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...,NaN,raw
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,NaN,not_saved,missed
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...,NaN,raw
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...,NaN,raw
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...,NaN,raw
...,...,...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...,saved,missed
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...,saved,missed
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...,saved,missed
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...,saved,missed


**Number of saved files in events folder**

In [12]:
events_files = []
regex = r"^(\d{4})-(\d{1,2})-(\d{1,2})"

for folder_id in events_folder_id_dict.values():
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
    files = results.get("files", [])
    next_page_token = results.get("nextPageToken")
    while next_page_token:
        results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
        files.extend(results.get("files", []))
        next_page_token = results.get("nextPageToken")
    for file in files:
        file_name = file.get('name')
        result = re.sub(regex, '', file_name)
        events_files.append(file_name.replace(result,''))

In [13]:
len(events_files)

5016

In [14]:
events_days = list(dict.fromkeys(events_files))
len(events_days)

1835

In [22]:
mask = ~df['date'].isin(events_files)
df.loc[mask, 'has_event'] = 'no'

In [24]:
df.has_event.value_counts()

yes    1835
no     1093
Name: has_event, dtype: int64

In [27]:
df

,issue,url,date,pdf_link,status,GD,has_event
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...,NaN,raw,no
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,NaN,not_saved,missed,no
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...,NaN,raw,no
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...,NaN,raw,no
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...,NaN,raw,no
...,...,...,...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...,saved,missed,no
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...,saved,missed,no
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...,saved,missed,no
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...,saved,missed,no
